In [1]:
import h3
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

In [2]:
# read geojson file
f = open('../gangnamGu.geojson')
data = json.load(f)
f.close()

In [3]:
# because h3 get coordinates as reversed order of geojson file. I have to reverse the order of coordinates prior to get hexagons
new_coords = [[coord[1],coord[0]] for coord in data['features'][0]['geometry']['coordinates'][0]]

In [4]:
# generate a new geojson file with the reversed coordinates
new_geojson = {'type':'Polygon', 'coordinates':[new_coords]}

In [5]:
# get indices of h3 hexagons from a new geojson 
hexagons = list(h3.polyfill(new_geojson, res=10))
print(hexagons)

['8a30e1c8084ffff', '8a30e1c85aaffff', '8a30e1ca126ffff', '8a30e1c88797fff', '8a30e1ca4257fff', '8a30e1ca3c27fff', '8a30e1ca0b57fff', '8a30e1c8e817fff', '8a30e1ca2acffff', '8a30e1c8531ffff', '8a30e1c8469ffff', '8a30e1ca335ffff', '8a30e1c8ea87fff', '8a30e1c8eaaffff', '8a30e1ca4adffff', '8a30e1ca3577fff', '8a30e1ca510ffff', '8a30e1ca1087fff', '8a30e1ca5c4ffff', '8a30e1ca020ffff', '8a30e1ca32d7fff', '8a30e1ca0997fff', '8a30e1c88d37fff', '8a30e1c81d97fff', '8a30e1ca18effff', '8a30e1c81b07fff', '8a30e1c888f7fff', '8a30e1c848c7fff', '8a30e1c8528ffff', '8a30e1c80b8ffff', '8a30e1c81067fff', '8a30e1cae69ffff', '8a30e1ca54f7fff', '8a30e1c8090ffff', '8a30e1ca34effff', '8a30e1ca0c77fff', '8a30e1c80a37fff', '8a30e1c8139ffff', '8a30e1ca1d5ffff', '8a30e1c8414ffff', '8a30e1cae12ffff', '8a30e1c8ec0ffff', '8a30e1ca3617fff', '8a30e1ca1b97fff', '8a30e1c8cb07fff', '8a30e1c88d57fff', '8a30e1c85507fff', '8a30e1ca174ffff', '8a30e1ca5047fff', '8a30e1c8855ffff', '8a30e1c85967fff', '8a30e1c8e92ffff', '8a30e1c840

In [6]:
# get centerpoints from the indices
hexagons_centerpoints = [[index, h3.h3_to_geo(index)[1],h3.h3_to_geo(index)[0]] for index in hexagons]
print(hexagons_centerpoints)

[['8a30e1c8084ffff', 127.06169487376121, 37.476326766386855], ['8a30e1c85aaffff', 127.06762554458656, 37.49352218368782], ['8a30e1ca126ffff', 127.05960707917032, 37.5230745360948], ['8a30e1c88797fff', 127.11942859296818, 37.46353574717951], ['8a30e1ca4257fff', 127.02237456230182, 37.52569849641242], ['8a30e1ca3c27fff', 127.03831659200605, 37.5012808369898], ['8a30e1ca0b57fff', 127.03251575440765, 37.52157594372449], ['8a30e1c8e817fff', 127.09371587818272, 37.46963971297612], ['8a30e1ca2acffff', 127.0346971583553, 37.49826982499011], ['8a30e1c8531ffff', 127.0703205743695, 37.4899208355774], ['8a30e1c8469ffff', 127.05039630724339, 37.471021834608884], ['8a30e1ca335ffff', 127.05569843378179, 37.50317634899506], ['8a30e1c8ea87fff', 127.1018369731153, 37.468206565946055], ['8a30e1c8eaaffff', 127.10229259685468, 37.46916880869764], ['8a30e1ca4adffff', 127.01854951359091, 37.529233959324145], ['8a30e1ca3577fff', 127.04281420881595, 37.496843495029225], ['8a30e1ca510ffff', 127.02984882826587, 

In [7]:
hexagons_polygons =[Polygon([(coords[1], coords[0]) for coords in h3.h3_to_geo_boundary(index)]) for index in hexagons]
print(hexagons_polygons[:5])

[<shapely.geometry.polygon.Polygon object at 0x0000018B080BF670>, <shapely.geometry.polygon.Polygon object at 0x0000018B080BF5E0>, <shapely.geometry.polygon.Polygon object at 0x0000018B080BF3D0>, <shapely.geometry.polygon.Polygon object at 0x0000018B080BFDF0>, <shapely.geometry.polygon.Polygon object at 0x0000018B080BB8B0>]


In [8]:
# create a dataframe with the indices and centerpoints
df_hexagons = pd.DataFrame(hexagons_centerpoints, columns =['index', 'x','y'])

In [9]:
df_hexagons.head()

,index,x,y
0,8a30e1c8084ffff,127.061695,37.476327
1,8a30e1c85aaffff,127.067626,37.493522
2,8a30e1ca126ffff,127.059607,37.523075
3,8a30e1c88797fff,127.119429,37.463536
4,8a30e1ca4257fff,127.022375,37.525698


In [10]:
# convert the dataframe into a geodataframe with geometries
gdf_hexagons = gpd.GeoDataFrame(df_hexagons, geometry = hexagons_polygons)

In [11]:
gdf_hexagons.head()

,index,x,y,geometry
0,8a30e1c8084ffff,127.061695,37.476327,"POLYGON ((127.06230 37.47605, 127.06222 37.476..."
1,8a30e1c85aaffff,127.067626,37.493522,"POLYGON ((127.06823 37.49324, 127.06815 37.493..."
2,8a30e1ca126ffff,127.059607,37.523075,"POLYGON ((127.06021 37.52280, 127.06014 37.523..."
3,8a30e1c88797fff,127.119429,37.463536,"POLYGON ((127.12003 37.46326, 127.11996 37.463..."
4,8a30e1ca4257fff,127.022375,37.525698,"POLYGON ((127.02298 37.52542, 127.02290 37.526..."


In [12]:
# export the geodataframe into a geojson file 
gdf_hexagons.to_file('../hexagon_res_10/hexagons.geojson', encoding='utf8', driver='GeoJSON')